In [ ]:
%%timeit x = range(10000)
max(x)

In [ ]:
%timeit (i for i in range(10*1000000))

In [1]:
import numpy as np
import math
import random

import numba
from numba import jit
from numba import njit
from numba import int64








def kth_non_anti_diagonal(ray,row,col, edge, value):
    k = col - row
    if (k != 0):
        ray[kth_diag_indices(edge, k)] += value
    else:
        ray[np.diag_indices(edge)] += value

def kth_diag_indices(edge, k):
    rowidx, colidx = np.diag_indices(edge)
    colidx = colidx.copy()  # rowidx and colidx share the same buffer

    if k > 0:
        colidx += k  # use the diagonal k columns to the right of the major
    else:
        rowidx -= k # use the diagnoal k rows below the major diagonal
    k = np.abs(k)
    return rowidx[:-k], colidx[:-k] 

def kth_anti_diagonal(ray,row,col,edge,value):

    fliplr = np.fliplr(ray)
    kprime = (edge-1-row-col)

    if (kprime != 0):
        fliplr[kth_diag_indices(edge, kprime)] += value
    else:
        fliplr[np.diag_indices(edge)] += value
        
        
        

def jittish_kth_anti_diagonal(ray,row,col,edge,value):
    fliplr = np.fliplr(ray)
    jit_kth_anti_diagonal(fliplr,row,col,edge,value)
    
@njit(parallel = True, fastmath = True)
def jit_kth_anti_diagonal(ray,row,col,edge,value):

    kprime = (edge-1-row-col)

    if (kprime != 0):

        rowidx,colidx = jit_kth_diag_indices(edge, kprime)
        for i in prange(len(rowidx)):
            ray[rowidx[i], colidx[i]] += value
            
    else:

        for i in prange(edge):
            ray[i,i] += value

    
        
'''
@jit(nopython = True)
def jit_kth_non_anti_diagonal(ray,row,col,edge,value):
    k = col - row
    if (k != 0):
   --->     ray[jit_kth_diag_indices(edge, k)] += value
        #np.add.at(ray, kth_diag_indices(ray, k), value) #NO! takes 5x longer!!
    else:
        ray[np.diag_indices_from(ray)] += value
        
---> JIT does not like the advanced indexing SOOOO:
     put it back into a for loop and jitify it.
'''
@jit(nopython = True)
def jit_kth_non_anti_diagonal(ray,row,col,edge,value):

    k = col - row
    if (k != 0):
        rowidx,colidx = jit_kth_diag_indices(edge, k)
        
        for i in range(len(rowidx)):
            ray[rowidx[i], colidx[i]] += value
    else:
        for i in range(edge):
            ray[i,i] += value
        



@njit(parallel = True)
def jit_kth_diag_indices(edge, k):
    rowidx = np.empty(edge, dtype = int64)
    colidx = np.empty(edge, dtype = int64)

    if k > 0:
        for i in range(edge):
            rowidx[i] = i
            colidx[i] = i + k # use the diagonal k columns to the right of the major
    else:
        for i in range(edge):
            rowidx[i] = i -k # use the diagnoal k rows below the major diagonal
            colidx[i] = i
            
    k = np.abs(k)
    return (rowidx[:-k], colidx[:-k])


'''
@jit(nopython = True)
def jit_diag_indices(edge):
    rows = np.empty(edge, dtype = int64)
    cols = np.empty(edge, dtype = int64)
    for square in range(edge):
        rows[square] = square
        cols[square] = square
    return(rows, cols)
    
---> same advance indexing problem so replaced with an inline for loop


'''





        








''
@jit(nopython = True)
def jit_flip_indices_lr(edge,size):
    ''' 
    We do not need or want to flip the array
    We just want to return the flipped INDICES
    We then can apply jit_kth_diag_indices to them.
    
    '''
    newrow = np.zeros(size, dtype = int64)
    newcol = np.zeros(size, dtype = int64)
    for row in range(size):
        for col in range(edge):
            newrow[row] = (row%edge)
            newcol[row] = (edge - newrow[row] -1)
          
            
def diag_tester(edge):
    ray = np.zeros((edge,edge), dtype = int)
    
    for row in range(edge):
        for column in range(edge):
            ray = np.zeros((edge,edge), dtype = int)
            set_diagonal_conflicts(ray,row,column, edge, 1)
            
            jray = np.zeros((edge,edge), dtype = int)
            jit_set_diagonal_conflicts(jray,row,column, edge, 1)
            
            if (np.all(ray == jray) != True):
                print(f'Failed at Row: {row}; Col: {column}')
    print('Done')
            
            

    
            
            

def set_diagonal_conflicts(conflicts_array, my_row, my_column, edge, value_to_add_or_sub):
    # NON anti-diagonal;

    kth_non_anti_diagonal(conflicts_array, my_row, my_column, edge, value_to_add_or_sub)
    conflicts_array[my_row, my_column] += -value_to_add_or_sub

    # ANTI-diagonal

    kth_anti_diagonal(conflicts_array, my_row, my_column, edge, value_to_add_or_sub)
    conflicts_array[my_row, my_column] += -value_to_add_or_sub   
    
    
def jit_set_diagonal_conflicts(conflicts_array, my_row, my_column, edge, value_to_add_or_sub):

    # NON anti-diagonal
    jit_kth_non_anti_diagonal(conflicts_array, my_row, my_column, edge, value_to_add_or_sub)
    conflicts_array[my_row, my_column] += -value_to_add_or_sub

    # ANTI-diagonal
    jittish_kth_anti_diagonal(conflicts_array, my_row, my_column, edge, value_to_add_or_sub)
    conflicts_array[my_row, my_column] += -value_to_add_or_sub  
            



edge = 20000


row = 0
col = 1

ray = np.zeros((edge,edge), dtype = int)
#columns = np.random.randint(0, edge, edge)

%timeit kth_non_anti_diagonal(ray, row, col, edge, 1)
%timeit jit_kth_non_anti_diagonal(ray, row, col, edge, 1)
print("\n")

%timeit kth_non_anti_diagonal(ray, row, col, edge, 1)
%timeit jit_kth_non_anti_diagonal(ray, row, col, edge, 1)
print("\n")

%timeit kth_non_anti_diagonal(ray, row, col, edge, 1)
%timeit jit_kth_non_anti_diagonal(ray, row, col, edge, 1)
print("\n")

%timeit kth_non_anti_diagonal(ray, row, col, edge, 1)
%timeit jit_kth_non_anti_diagonal(ray, row, col, edge, 1)
print("\n")
    

    


1.25 ms ± 387 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
895 µs ± 239 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


1.92 ms ± 631 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
374 µs ± 27 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


1.26 ms ± 139 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
376 µs ± 27.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


1.39 ms ± 160 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
474 µs ± 91.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)




In [3]:
numba.__version__


'0.40.0'

In [5]:
import numpy as np
import numba

foo = np.array(1, dtype = int)
print(np.ndim(foo))

print(np.isscalar(foo))

print(foo.item())


0
False
1
